# Script to compute Panoptic Quality (PQ) metric for each image of each participant and to store results in an excel file


## Created by Ruchika Verma

This code will generate an excel file containing multiple sheets (named as participants team-names). Each sheet will save image names in rows and respective PQ metrics in columns.

`Epithelial` = column 1, `Lymphocyte`  = column 2, `Neutrophil` = column 3, `Macrophage` = column 4

Note: This code will work if n-ary masks are stored in both ground truth and predicted path. If the mask is stored as binary, it will first convert it into n-ary and then compute PQ metric
 
Please run n-ary mask generation code from [here](https://github.com/ruchikaverma-iitg/MoNuSAC/blob/master/n-ary_mask_generation.ipynb) to see ground truth and predicted masks format.

The [code](https://github.com/ruchikaverma-iitg/MoNuSAC/blob/master/n-ary_mask_generation.ipynb) will save masks to compute PQ metric as given below:<br>
-`Folder` -> Patient name <br>
-`Sub-folder` -> Sub-images under each patient<br>
-`Sub-Sub-folder` -> Annotated cell-type on each sub-image which contains n-ary masks (saved as mat file)

### Input
- ground_truth_path: Path to read ground truth masks from <br>
- Predicted_path: Path to read participant folders from <br>

### Output
An excel file with name `MoNuSAC-testing-PQ.xls` will store on the given ground_truth_path

### Reference
Kirillov, A., He, K., Girshick, R., Rother, C., & Dollár, P. (2019). Panoptic segmentation. In Proceedings of the IEEE conference on computer vision and pattern recognition (pp. 9404-9413).

In [1]:
import os
import numpy as np
import glob
import cv2
import scipy.io as sio
from PIL import Image
import scipy
import scipy.ndimage
import xlwt 
from xlwt import Workbook 
import re
import tifffile as tiff

In [2]:
# Compute Panoptic quality metric for each image
def Panoptic_quality(ground_truth_image,predicted_image):
    TP = 0
    FP = 0
    FN = 0
    sum_IOU = 0
    matched_instances = {}# Create a dictionary to save ground truth indices in keys and predicted matched instances as velues
                        # It will also save IOU of the matched instance in [indx][1]

    # Find matched instances and save it in a dictionary
    for i in np.unique(ground_truth_image):
        if i == 0:
            pass
        else:
            temp_image = np.array(ground_truth_image)
            temp_image = temp_image == i
            matched_image = temp_image * predicted_image
        
            for j in np.unique(matched_image):
                if j == 0:
                    pass
                else:
                    pred_temp = predicted_image == j
                    intersection = sum(sum(temp_image*pred_temp))
                    union = sum(sum(temp_image + pred_temp))
                    IOU = intersection/union
                    if IOU> 0.5:
                        matched_instances [i] = j, IOU 
                        
    # Compute TP, FP, FN and sum of IOU of the matched instances to compute Panoptic Quality               
                        
    pred_indx_list = np.unique(predicted_image)#Find all predicted instances
    pred_indx_list = np.array(pred_indx_list[1:])#Remove 0 from the predicted instances

    # Loop on ground truth instances
    for indx in np.unique(ground_truth_image):
        if indx == 0:
            pass
        else:
            if indx in matched_instances.keys():
                pred_indx_list = np.delete(pred_indx_list, np.argwhere(pred_indx_list == matched_instances[indx][0]))
                TP = TP+1
                sum_IOU = sum_IOU+matched_instances[indx][1]
            else:
                FN = FN+1
    FP = len(np.unique(pred_indx_list))
    PQ = sum_IOU/(TP+0.5*FP+0.5*FN)
    
    return PQ

In [3]:
ground_truth_base = 'MoNuSAC_masks' #Ground truth path to read data from
Predicted_path = 'predicted_masks' #Path to read predicted outcomes from
predicted_files = glob.glob(os.path.join(Predicted_path, '*.tif'))
# import os
# os.chdir(ground_truth_path)

In [4]:
participants_folders=glob.glob(Predicted_path+'/**')
participants_folders

# participants_folders = participants_folders[2:3]
participants_folders

['predicted_masks\\pred_0.tif',
 'predicted_masks\\pred_1.tif',
 'predicted_masks\\pred_10.tif',
 'predicted_masks\\pred_11.tif',
 'predicted_masks\\pred_12.tif',
 'predicted_masks\\pred_13.tif',
 'predicted_masks\\pred_14.tif',
 'predicted_masks\\pred_15.tif',
 'predicted_masks\\pred_16.tif',
 'predicted_masks\\pred_17.tif',
 'predicted_masks\\pred_18.tif',
 'predicted_masks\\pred_19.tif',
 'predicted_masks\\pred_2.tif',
 'predicted_masks\\pred_20.tif',
 'predicted_masks\\pred_21.tif',
 'predicted_masks\\pred_22.tif',
 'predicted_masks\\pred_23.tif',
 'predicted_masks\\pred_24.tif',
 'predicted_masks\\pred_25.tif',
 'predicted_masks\\pred_26.tif',
 'predicted_masks\\pred_27.tif',
 'predicted_masks\\pred_28.tif',
 'predicted_masks\\pred_29.tif',
 'predicted_masks\\pred_3.tif',
 'predicted_masks\\pred_30.tif',
 'predicted_masks\\pred_31.tif',
 'predicted_masks\\pred_32.tif',
 'predicted_masks\\pred_33.tif',
 'predicted_masks\\pred_34.tif',
 'predicted_masks\\pred_35.tif',
 'predicted_ma

In [5]:
cell_types = ['Epithelial','Lymphocyte', 'Neutrophil','Macrophage']

In [6]:
files=glob.glob('./**/**')
len(files) #Ground Truth files

103

In [7]:
   
for participant_folder in participants_folders:
    print(participant_folder[64:])


In [8]:
# Workbook is created
wb = Workbook() 
sheet_name = 'Predictions'
ccbt = wb.add_sheet(sheet_name)

# Header schreiben
ccbt.write(0, 0, 'Patient ID')
for col, cell_type in enumerate(cell_types):
    ccbt.write(0, col+1, cell_type)

# --- Alle Prediction-Dateien durchgehen ---
predicted_files = glob.glob(os.path.join(Predicted_path, '*.tif'))

for row_idx, pred_path in enumerate(predicted_files):
    file_name = os.path.basename(pred_path)
    ccbt.write(row_idx+1, 0, file_name)

    # --- Ground Truth rekursiv finden ---
    file_number = re.search(r'\d+', file_name).group(0)

    # Ground Truth suchen, die diese Zahl enthält
    gt_matches = glob.glob(os.path.join(ground_truth_base, '**', f'*{file_number}*.tif'), recursive=True)

    if not gt_matches:
        print(f"Ground truth fehlt für {file_name}")
        for col in range(len(cell_types)):
            ccbt.write(row_idx+1, col+1, 0)
        continue

    gt_path = gt_matches[0]

    # --- TIFF einlesen ---
    ground_truth = tiff.imread(gt_path)
    predicted_mask = tiff.imread(pred_path)

    # --- Ambiguous Regions optional ---
    ambiguous_path = gt_path.replace('.tif', '_Ambiguous.tif')
    if os.path.exists(ambiguous_path):
        ambiguous_regions = tiff.imread(ambiguous_path)
        ambiguous_regions = 1 - (ambiguous_regions > 0)
        predicted_mask = predicted_mask * ambiguous_regions

    # --- Binary zu n-ary Mask, falls nötig ---
    if len(np.unique(predicted_mask)) == 2:
        predicted_mask, num_features = scipy.ndimage.label(predicted_mask)

    # --- Panoptic Quality berechnen ---
    PQ = Panoptic_quality(ground_truth, predicted_mask)  # Sollte Liste mit 4 Werten für cell_types liefern
    print(f"{file_name}: PQ = {PQ}")

    # --- PQ-Werte ins Excel eintragen ---
    for col in range(len(cell_types)):
        ccbt.write(row_idx+1, col+1, PQ)

# --- Excel speichern ---
wb.save('results.xls')
print("Excel-Datei 'results.xls' wurde erstellt.")

pred_0.tif: PQ = 0.0
pred_1.tif: PQ = 0.0
pred_10.tif: PQ = 0.0
pred_11.tif: PQ = 0.0
pred_12.tif: PQ = 0.0
pred_13.tif: PQ = 0.0
pred_14.tif: PQ = 0.0
pred_15.tif: PQ = 0.0
pred_16.tif: PQ = 0.0
pred_17.tif: PQ = 0.0
pred_18.tif: PQ = 0.0
pred_19.tif: PQ = 0.0
pred_2.tif: PQ = 0.0
pred_20.tif: PQ = 0.0
pred_21.tif: PQ = 0.0
pred_22.tif: PQ = 0.0
pred_23.tif: PQ = 0.0
pred_24.tif: PQ = 0.0
pred_25.tif: PQ = 0.0
pred_26.tif: PQ = 0.0
pred_27.tif: PQ = 0.0
pred_28.tif: PQ = 0.0
pred_29.tif: PQ = 0.0
pred_3.tif: PQ = 0.0
pred_30.tif: PQ = 0.0
pred_31.tif: PQ = 0.0
pred_32.tif: PQ = 0.0
pred_33.tif: PQ = 0.0
pred_34.tif: PQ = 0.0
pred_35.tif: PQ = 0.0
pred_36.tif: PQ = 0.0
pred_37.tif: PQ = 0.0
pred_38.tif: PQ = 0.0
pred_39.tif: PQ = 0.0
pred_4.tif: PQ = 0.0
pred_40.tif: PQ = 0.0
pred_41.tif: PQ = 0.0
pred_5.tif: PQ = 0.0
pred_6.tif: PQ = 0.0
pred_7.tif: PQ = 0.0
pred_8.tif: PQ = 0.0
pred_9.tif: PQ = 0.0
Excel-Datei 'results.xls' wurde erstellt.
